In [5]:
import requests
import re
from bs4 import BeautifulSoup, Comment

import json
import numpy as np
import pandas as pd
import time

from pymongo import MongoClient

In [2]:
webpage = requests.get('https://www.theonion.com/religious-conservatives-argue-adam-and-eve-would-never-1833267416')

In [3]:
soup = BeautifulSoup(webpage.text, 'lxml')

In [23]:
web = soup.find_all('div', {'class':'post-content entry-content js_entry-content '})[0].text

In [24]:
web

'HOUSTON—In what they described as scriptural evidence of the right to bear arms, leading figures among the religious right gathered Wednesday to issue a statement arguing that Adam and Eve would never have been banished from the Garden of Eden if they had owned guns. “Just imagine: If Adam and Eve had carried firearms and stood their ground against God, they would have been able to eat the fruit from the Tree of Knowledge in peace, and He could never have forced them to leave paradise,” said Pastor Hugh Peters of Houston’s Second Baptist Church, explaining how the entire course of human history would have been altered for the better if the first man and woman had taken the simple precaution of keeping a semiautomatic weapon at the ready for use during emergencies. “God was trespassing on their property, pure and simple. He had absolutely no right to force them from their home. Had Eve been able to open-carry a handgun, maybe tying it to her hip with a vine or something, God would have

In [6]:
links = requests.get('https://www.theonion.com/c/news-in-brief')

In [7]:
soup2 = BeautifulSoup(links.text, 'lxml')

In [8]:
soup2

<!DOCTYPE html>
<html data-reactroot=""><head><meta content="P13oAn8o8LB6FVCJWKsHWXvxfbR-SJrmvox6EgULpUs" name="google-site-verification"/><meta name="ir-site-verification-token" value="-1270174611"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0,maximum-scale=10.0" name="viewport"/><meta content="INDEX, FOLLOW" name="ROBOTS"/><title>News in Brief -  America's Finest News Source | The Onion </title><link href="https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_80,q_80,w_80/eti2h1r4wg0bqxctxenl.png" rel="shortcut icon" type="image/png"/><link href="https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_200,q_80,w_200/eti2h1r4wg0bqxctxenl.png" rel="apple-touch-icon" type="image/png"/><meta content="https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_80,q_80,w_80/eti2h1r4wg0bqxctxenl.png" name="msapplication-square70x70logo"/><meta content="https://i.kin

In [151]:
web3 = soup2.find_all('div')

In [152]:
len(web3)

285

In [9]:
linkers = soup2.find_all('h1')

In [10]:
linkers[0].a['href']

'https://politics.theonion.com/beto-o-rourke-announces-he-starting-obama-cover-campaig-1833305932'

In [11]:
len(linkers)

20

In [18]:
test = ['https://politics.theonion.com/beto-o-rourke-announces-he-starting-obama-cover-campaig-1833305932']

In [25]:
links = requests.get('https://www.theonion.com/c/news-in-brief')
soup2 = BeautifulSoup(links.text, 'lxml')
nxt = scrape(soup2)
    

{'https://politics.theonion.com/beto-o-rourke-announces-he-starting-obama-cover-campaig-1833305932': 'EL PASO, TX—Revealing plans to “put his own spin” on beloved stump speeches and talking points, Beto O’Rourke announced Thursday that he was starting a Barack Obama cover campaign. “I’ve always loved Barack’s early stuff from back in ’08, even ’04, and I think diehard fans will go crazy when I cover all his greatest hits,” said the 46-year-old White House hopeful, clarifying that he and his campaign aides had spent several months “just going through Barack’s catalog” to memorize the former president’s platform. “Obama’s style always really resonated with me, and honestly, what’s the point in struggling to come up with something new when you can just give people what they want. I’m going to cover some of Barack’s most well-known campaign speeches, putting my own small twists on prison reform and healthcare—they’ll be way heavier, faster, and louder.” At press time, O’Rourke revealed tha

{'https://local.theonion.com/tennis-instructor-mentoring-young-player-sees-potential-1833286829': 'LOCK HAVEN, PA—Calling him the most promising recruit he’s seen in a decade of coaching, tennis instructor Thomas Petrov confirmed Thursday that he sees real potential in his student Aiden McDavid’s family income. “From his expensive Babolat racket to the brand-new Nikes, I can tell this kid’s parents have everything I’m looking for,” said Petrov, noting that he saw promise in Aiden the moment his parents dropped him off at practice in a brand-new Mercedes Benz sedan. “I’ve mentored a lot of kids throughout the years, and I can tell that Aiden’s special—his parents aren’t just some lawyers or mid-level executives, they’re running a hedge fund. Some kids show hints of potential, but these people are rocking Cartier watches and vacationing in Italy. They’re the real deal.” At press time, Petrov convinced Aiden’s parents that he could be something really special after watching the teen pack 

{'https://politics.theonion.com/new-iowa-poll-finds-majority-of-democrats-would-vote-fo-1833237251': 'DES MOINES, IA—Offering new insight into the preferences of Democratic voters, a poll released Tuesday by The Des Moines Register and CNN found that nearly 60 percent of likely Iowa caucus-goers would support a presidential candidate with the name Bobby Cheeseburger. “As many as 47 percent of those surveyed indicated that they’d vote for somebody called Bobby Cheeseburger in a heartbeat without even needing to know all his policy positions,” said pollster Emily Jones, telling reporters that almost a third of Iowans believed a guy named Bobby Cheeseburger could use his homespun wisdom and commitment to American values to bring real change to Washington. “Over half of respondents said they believe that a Bobby Cheeseburger candidacy would stand a strong chance of defeating Trump in the general election by bridging the gap between centrists and liberals as well as appealing to Republicans

In [26]:
def scrape(soup_object):
    '''
    INPUT:
        -soup_object: BeautifulSoup object
    OUTPUT:
        -Dictioary to mongo db
            Dictionary is in the form of url:article content.
            Each article is stored in mongo as its own dict
        -string: link for more articles button at bottom of the page
    '''
    linkers = soup_object.find_all('h1')
    linkers[0].a['href']
    https = []
    for idx, x in enumerate(linkers):
        https.append(linkers[idx].a['href'])
    for link in https:
        onion = {}
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'lxml')
        content = soup.find_all('div', {'class':'post-content entry-content js_entry-content '})[0].text
        onion[link] = content[0:-13]
        print(onion)
        time.sleep(20)
        count += 1
    button = soup2.find_all('div', {'class': 'sc-1uzyw0z-0 kiwkfc'})
    more_button = button[0].a['href']
    return 'https://www.theonion.com/c/news-in-brief' + more_button
    

In [168]:
button = soup2.find_all('div', {'class': 'sc-1uzyw0z-0 kiwkfc'})
more_button = button[0].a['href']
next_page = 'https://www.theonion.com/c/news-in-brief' + more_button

In [174]:
more_button = button[0].a['href']

In [175]:
next_page = 'https://www.theonion.com/c/news-in-brief' + more_button

In [176]:
next_page

'https://www.theonion.com/c/news-in-brief?startTime=1552323060200'